In [ ]:
import sys, os

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow.keras.layers as layers
import tensorflow as tf
import numpy as np
import os

In [ ]:
# feature extractor
mobilenet_v2 = MobileNetV2(input_shape=(224,224,3), weights="imagenet", include_top=False)
tf.keras.utils.plot_model(mobilenet_v2, show_shapes=True)


In [ ]:
output_tensor = mobilenet_v2.get_layer("block_5_add").output
feature_extractor = tf.keras.Model(inputs=[mobilenet_v2.input], outputs=[output_tensor])
feature_extractor.summary()

In [ ]:
def build_mobilenet_block(num_filter, strides=(2,2)):
    block = tf.keras.Sequential([tf.keras.layers.Conv2D(num_filter, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.DepthwiseConv2D(3, strides=strides, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.Conv2D(num_filter, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU()])
    return block


neck1 = build_mobilenet_block(512, (1,1)) #28,28
neck2 = build_mobilenet_block(512, (1,1)) #28,28
neck3 = build_mobilenet_block(512, (1,1)) #28,28
block1 = build_mobilenet_block(256) #14,14
block2 = build_mobilenet_block(256) # 7, 7
block3 = build_mobilenet_block(256) # 4, 4
block4 = build_mobilenet_block(256) # 2, 2
block5 = build_mobilenet_block(256) # 1, 1

In [ ]:
input_layer = tf.keras.Input(shape=(224,224,3))
x = feature_extractor(input_layer)
x = neck1(x)
x = neck2(x)
x = neck3(x)
y1 = block1(x)
y2 = block2(y1)
y3 = block3(y2)
y4 = block4(y3)
y5 = block5(y4)

In [ ]:
'''
from utils.anchor import generate_retina_boxes
anchor_param = {"ratios": [0.5, 1, 2, 3, 5],
                "scales": [1.0, 1.33, 1.66],
                           "fm_sizes": [32, 16, 8, 4, 2, 1],
                           "image_size": 128} #anchor parameters
anchors = generate_retina_boxes(anchor_param)
'''

In [ ]:
for i in [x, y1,y2,y3,y4,y5]:
    print(i.shape)

In [ ]:
def build_head_network(filter=3):
    head_network = tf.keras.Sequential()
    head_network.add(tf.keras.layers.Conv2D(256, kernel_size=filter, activation='relu' ,padding='same'))
    head_network.add(tf.keras.layers.BatchNormalization())
    head_network.add(tf.keras.layers.ReLU())
    return head_network

#conf header
anchor_types = 15
num_classes = 4
confs = []

conf = build_head_network(filter=3)(x)
conf = layers.Conv2D(anchor_types * num_classes, kernel_size=3, padding='same')(conf)
conf = tf.reshape(conf, [-1, 28*28*anchor_types,num_classes])
confs.append(conf)

conf = build_head_network(filter=3)(y1)
conf = layers.Conv2D(anchor_types * num_classes, kernel_size=3, padding='same')(conf)
conf = tf.reshape(conf, [-1, 14*14*anchor_types,num_classes])
confs.append(conf)

conf = build_head_network(filter=3)(y2)
conf = layers.Conv2D(anchor_types * num_classes, kernel_size=3, padding='same')(conf)
conf = tf.reshape(conf, [-1, 7*7*anchor_types,num_classes])
confs.append(conf)

conf = build_head_network(filter=3)(y3)
conf = layers.Conv2D(anchor_types * num_classes, kernel_size=3, padding='same')(conf)
conf = tf.reshape(conf, [-1, 4*4*anchor_types,num_classes])
confs.append(conf)

conf = build_head_network(filter=3)(y4)
conf = layers.Conv2D(anchor_types * num_classes, kernel_size=3, padding='same')(conf)
conf = tf.reshape(conf, [-1, 2*2*anchor_types,num_classes])
confs.append(conf)

conf = build_head_network(filter=1)(y5)
conf = layers.Conv2D(anchor_types * num_classes, kernel_size=3, padding='same')(conf)
conf = tf.reshape(conf, [-1, 1*1*anchor_types,num_classes])
confs.append(conf)



#location header
anchor_types = 15
coords = 4
locs = []

loc = build_head_network(filter=3)(x)
loc = layers.Conv2D(anchor_types * coords, kernel_size=3, padding='same')(loc)
loc = tf.reshape(loc, [-1, 28*28*anchor_types, coords])
locs.append(loc)

loc = build_head_network(filter=3)(y1)
loc = layers.Conv2D(anchor_types * coords, kernel_size=3, padding='same')(loc)
loc = tf.reshape(loc, [-1, 14*14*anchor_types, coords])
locs.append(loc)

loc = build_head_network(filter=3)(y2)
loc = layers.Conv2D(anchor_types * coords, kernel_size=3, padding='same')(loc)
loc = tf.reshape(loc, [-1, 7*7*anchor_types, coords])
locs.append(loc)

loc = build_head_network(filter=3)(y3)
loc = layers.Conv2D(anchor_types * coords, kernel_size=3, padding='same')(loc)
loc = tf.reshape(loc, [-1, 4*4*anchor_types, coords])
locs.append(loc)

loc = build_head_network(filter=3)(y4)
loc = layers.Conv2D(anchor_types * coords, kernel_size=3, padding='same')(loc)
loc = tf.reshape(loc, [-1, 2*2*anchor_types, coords])
locs.append(loc)

loc = build_head_network(filter=1)(y5)
loc = layers.Conv2D(anchor_types * coords, kernel_size=3, padding='same')(loc)
loc = tf.reshape(loc, [-1, 1*1*anchor_types, coords])
locs.append(loc)

confs = tf.concat(confs, axis=-2)
locs = tf.concat(locs, axis=-2)

In [ ]:
model = tf.keras.Model(inputs=[input_layer], outputs=[confs,locs])
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True)

In [ ]:
model.save("SSD-mobilenet-224-imagenet.h5")